In [1]:
# imports
import pandas as pd 
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn import metrics
from sklearn.metrics import confusion_matrix

#read in datasets
full_dataset = pd.read_csv("data/model_data.csv", index_col = 0)
full_dataset = full_dataset[np.isfinite(full_dataset['ARRIVAL_DELAY'])]

og_data = pd.read_csv("data/flights.csv")

print (full_dataset.shape)
print og_data.shape

/Users/nnbenavides/.local/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (7,8) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


(88500, 19)
(5819079, 31)


In [2]:
full_dataset.head()

,ARRIVAL_DELAY,DAY,MONTH,DAY_OF_WEEK,AIRLINE,ORIGIN_AIRPORT,DESTINATION_AIRPORT,SCHEDULED_DEPARTURE,TAIL_NUMBER,TEMPERATURE,ICON,HUMIDITY,SUMMARY,WIND_SPEED,VISIBILITY,PRECIP_INTENSITY,PRECIP_PROB,MODEL,YEAR
135485,0,28,7,2,AA,ORD,CLT,7,N939UW,3,clear-day,3,Clear,6,5,1,1,757-2B7,1994
64198,0,10,7,5,EV,ORD,MEM,12,N12552,2,partly-cloudy-day,2,Partly Cloudy,3,5,1,1,EMB-145LR,2002
138728,0,25,9,5,UA,ORD,RSW,7,N26232,2,clear-day,4,Clear,6,4,1,1,737-824,1999
21287,0,25,5,1,OO,ORD,BOI,12,N124SY,0,wind,3,Breezy and Overcast,8,4,1,4,ERJ 170-200 LR,2014
127549,0,16,3,1,UA,ORD,SMF,19,N36447,3,clear-night,2,Clear,7,5,1,1,737-924ER,2012


In [3]:
print list(og_data.columns)
og_data.head()

['YEAR', 'MONTH', 'DAY', 'DAY_OF_WEEK', 'AIRLINE', 'FLIGHT_NUMBER', 'TAIL_NUMBER', 'ORIGIN_AIRPORT', 'DESTINATION_AIRPORT', 'SCHEDULED_DEPARTURE', 'DEPARTURE_TIME', 'DEPARTURE_DELAY', 'TAXI_OUT', 'WHEELS_OFF', 'SCHEDULED_TIME', 'ELAPSED_TIME', 'AIR_TIME', 'DISTANCE', 'WHEELS_ON', 'TAXI_IN', 'SCHEDULED_ARRIVAL', 'ARRIVAL_TIME', 'ARRIVAL_DELAY', 'DIVERTED', 'CANCELLED', 'CANCELLATION_REASON', 'AIR_SYSTEM_DELAY', 'SECURITY_DELAY', 'AIRLINE_DELAY', 'LATE_AIRCRAFT_DELAY', 'WEATHER_DELAY']


,YEAR,MONTH,DAY,DAY_OF_WEEK,AIRLINE,FLIGHT_NUMBER,TAIL_NUMBER,ORIGIN_AIRPORT,DESTINATION_AIRPORT,SCHEDULED_DEPARTURE,...,ARRIVAL_TIME,ARRIVAL_DELAY,DIVERTED,CANCELLED,CANCELLATION_REASON,AIR_SYSTEM_DELAY,SECURITY_DELAY,AIRLINE_DELAY,LATE_AIRCRAFT_DELAY,WEATHER_DELAY
0,2015,1,1,4,AS,98,N407AS,ANC,SEA,5,...,408.0,-22.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
1,2015,1,1,4,AA,2336,N3KUAA,LAX,PBI,10,...,741.0,-9.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
2,2015,1,1,4,US,840,N171US,SFO,CLT,20,...,811.0,5.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
3,2015,1,1,4,AA,258,N3HYAA,LAX,MIA,20,...,756.0,-9.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
4,2015,1,1,4,AS,135,N527AS,SEA,ANC,25,...,259.0,-21.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
# select features for baseline model
#baseline_features = full_dataset[['AIRLINE','MONTH', 'DAY_OF_WEEK', 'DESTINATION_AIRPORT','SCHEDULED_DEPARTURE', 'MODEL', 'YEAR', 'TEMPERATURE', 'WIND_SPEED', 'VISIBILITY', 'PRECIP_PROB', 'PRECIP_INTENSITY', 'HUMIDITY']]
baseline_features = full_dataset[['AIRLINE','MONTH', 'DAY_OF_WEEK', 'DESTINATION_AIRPORT','SCHEDULED_DEPARTURE']]

In [5]:
#baseline_yx = full_dataset[['ARRIVAL_DELAY','AIRLINE','MONTH', 'DAY_OF_WEEK', 'DESTINATION_AIRPORT','SCHEDULED_DEPARTURE', 'MODEL', 'YEAR', 'TEMPERATURE', 'WIND_SPEED', 'VISIBILITY', 'PRECIP_PROB', 'PRECIP_INTENSITY', 'HUMIDITY']]
baseline_yx = full_dataset[['ARRIVAL_DELAY','AIRLINE','MONTH', 'DAY_OF_WEEK', 'DESTINATION_AIRPORT','SCHEDULED_DEPARTURE']]

In [6]:
encoded_target = baseline_yx[['ARRIVAL_DELAY']]

In [7]:
# encode non-numeric features
encoded_baseline_features = pd.concat([baseline_features.drop('AIRLINE', axis=1), pd.get_dummies(baseline_features['AIRLINE'])], axis=1)
#encoded_baseline_features = pd.concat([encoded_baseline_features, pd.get_dummies(baseline_features['MODEL'])], axis=1)
encoded_baseline_features = pd.concat([encoded_baseline_features, pd.get_dummies(baseline_features['DESTINATION_AIRPORT'])], axis=1)
encoded_baseline_features = encoded_baseline_features.drop('DESTINATION_AIRPORT', axis = 1)
#encoded_baseline_features= encoded_baseline_features.drop('MODEL', axis = 1)
#encoded_baseline_features = pd.concat([encoded_baseline_features, pd.get_dummies(baseline_features['YEAR'])], axis=1)
#encoded_baseline_features= encoded_baseline_features.drop('YEAR', axis = 1)



In [27]:
# split dataset and fit multiclass logistic regression model
train_xb, test_xb, train_yb, test_yb = train_test_split(encoded_baseline_features, encoded_target, train_size=0.8)

lr = LogisticRegression()
lr.fit(train_xb, train_yb)
lr_train_acc = metrics.accuracy_score(train_yb, lr.predict(train_xb))
lr_test_acc = metrics.accuracy_score(test_yb, lr.predict(test_xb))

# results
print ('baseline train acc: ' + str(lr_train_acc))
print ('baseline test acc: ' + str(lr_test_acc))
print ('baseline precision: ' + str(metrics.precision_score(test_yb, lr.predict(test_xb), average = 'weighted')))
print ('baseline recall: ' + str(metrics.recall_score(test_yb, lr.predict(test_xb), average = 'weighted')))

baseline train acc: 0.41552259887
baseline test acc: 0.409039548023
baseline precision: 0.408970936155
baseline recall: 0.409039548023


In [ ]:
# baseline confusion matrix
y_true = test_yb
y_pred = lr.predict(test_xb)
confusion = confusion_matrix(y_true, y_pred)
print confusion

In [ ]:
# binarized baseline
train_xb, test_xb, train_yb, test_yb = train_test_split(encoded_baseline_features, encoded_target, train_size=0.8)

lr = LogisticRegression()
lr.fit(train_xb, train_yb)
test_y_binarized = test_yb.as_matrix().tolist()
test_y_binarized = [1 if test_y_binarized[i][0] > 1 else 0 for i in range(len(test_y_binarized))]
pred_y_binarized = lr.predict(test_xb)
pred_y_binarized = [1 if pred_y_binarized[i] > 1 else 0 for i in range(len(pred_y_binarized))]


lr_train_acc = metrics.accuracy_score(train_yb, lr.predict(train_xb))
lr_test_acc = metrics.accuracy_score(test_y_binarized, pred_y_binarized)

# results
print ('baseline train acc: ' + str(lr_train_acc))
print ('baseline test acc: ' + str(lr_test_acc))
print ('baseline precision: ' + str(metrics.precision_score(test_y_binarized, pred_y_binarized, average = 'binary')))
print ('baseline recall: ' + str(metrics.recall_score(test_y_binarized, pred_y_binarized, average = 'binary')))

In [ ]:
# get DEPARTURE_DELAY for oracle model
full_dataset = pd.merge(full_dataset, og_data, on = ['DAY', 'MONTH', 'AIRLINE', 'DESTINATION_AIRPORT','TAIL_NUMBER'])
print full_dataset.shape

full_dataset = full_dataset.drop('ARRIVAL_DELAY_y', axis = 1)
full_dataset = full_dataset.drop('CANCELLED', axis = 1)
full_dataset = full_dataset.drop('CANCELLATION_REASON', axis = 1)
full_dataset = full_dataset.drop('AIR_SYSTEM_DELAY', axis = 1)
full_dataset = full_dataset.drop('SECURITY_DELAY', axis = 1)
full_dataset = full_dataset.drop('AIRLINE_DELAY', axis = 1)
full_dataset = full_dataset.drop('LATE_AIRCRAFT_DELAY', axis = 1)
full_dataset = full_dataset.drop('WEATHER_DELAY', axis = 1)
full_dataset = full_dataset.drop('YEAR_y', axis = 1)
full_dataset = full_dataset.drop('DAY_OF_WEEK_y', axis = 1)
full_dataset = full_dataset.drop('FLIGHT_NUMBER', axis = 1)
full_dataset = full_dataset.drop('ORIGIN_AIRPORT_y', axis = 1)
full_dataset = full_dataset.drop('SCHEDULED_DEPARTURE_y', axis = 1)
full_dataset = full_dataset.drop('DEPARTURE_TIME', axis = 1)
full_dataset = full_dataset.drop('TAXI_OUT', axis = 1)
full_dataset = full_dataset.drop('WHEELS_OFF', axis = 1)
full_dataset = full_dataset.drop('SCHEDULED_TIME', axis = 1)
full_dataset = full_dataset.drop('ELAPSED_TIME', axis = 1)
full_dataset = full_dataset.drop('AIR_TIME', axis = 1)
full_dataset = full_dataset.drop('DISTANCE', axis = 1)
full_dataset = full_dataset.drop('WHEELS_ON', axis = 1)
full_dataset = full_dataset.drop('TAXI_IN', axis = 1)
full_dataset = full_dataset.drop('SCHEDULED_ARRIVAL', axis = 1)
full_dataset = full_dataset.drop('ARRIVAL_TIME', axis = 1)
full_dataset = full_dataset.drop('DIVERTED', axis = 1)
full_dataset.columns = ['ARRIVAL_DELAY','DAY','MONTH','DAY_OF_WEEK','AIRLINE','ORIGIN_AIRPORT','DESTINATION_AIRPORT','SCHEDULED_DEPARTURE','TAIL_NUMBER','TEMPERATURE','ICON','HUMIDITY','SUMMARY','WIND_SPEED','VISIBILITY','PRECIP_INTENSITY','PRECIP_PROB','MODEL','YEAR','DEPARTURE_DELAY']

full_dataset.head()

In [ ]:
# select features for oracle model
oracle_yx = full_dataset[['ARRIVAL_DELAY','AIRLINE','MONTH', 'DAY_OF_WEEK', 'DESTINATION_AIRPORT','SCHEDULED_DEPARTURE', 'MODEL', 'YEAR', 'TEMPERATURE', 'WIND_SPEED', 'VISIBILITY', 'PRECIP_PROB', 'PRECIP_INTENSITY', 'HUMIDITY', 'DEPARTURE_DELAY']]
oracle_yx = oracle_yx.dropna()
print oracle_yx.shape

In [ ]:
# encode non-numeric features
oracle_yx = pd.concat([oracle_yx.drop('AIRLINE', axis=1), pd.get_dummies(oracle_yx['AIRLINE'])], axis=1)
oracle_yx = pd.concat([oracle_yx, pd.get_dummies(oracle_yx['MODEL'])], axis=1)
oracle_yx = oracle_yx.drop('MODEL', axis = 1)
oracle_yx = pd.concat([oracle_yx, pd.get_dummies(oracle_yx['DESTINATION_AIRPORT'])], axis=1)
oracle_yx = oracle_yx.drop('DESTINATION_AIRPORT', axis = 1)
oracle_y = oracle_yx[['ARRIVAL_DELAY']]
oracle_x = oracle_yx.drop('ARRIVAL_DELAY', axis = 1)
print oracle_yx.shape


In [ ]:
train_xo, test_xo, train_yo, test_yo = train_test_split(oracle_x, oracle_y, train_size=0.8)
lr_oracle = LogisticRegression()
lr_oracle.fit(train_xo, train_yo)
lr_train_acc = metrics.accuracy_score(train_yo, lr_oracle.predict(train_xo))
lr_test_acc = metrics.accuracy_score(test_yo, lr_oracle.predict(test_xo))

print ('oracle train acc: ' + str(lr_train_acc))
print ('oracle test acc: ' + str(lr_test_acc))
print ('oracle precision: ' + str(metrics.precision_score(test_yo, lr_oracle.predict(test_xo), average = 'weighted')))
print ('oracle recall: ' + str(metrics.recall_score(test_yo, lr_oracle.predict(test_xo), average = 'weighted')))

In [ ]:
# binarized oracle
lr_oracle = LogisticRegression()
lr_oracle.fit(train_xo, train_yo)

test_y_binarized = test_yo.as_matrix().tolist()
test_y_binarized = [1 if test_y_binarized[i][0] > 1 else 0 for i in range(len(test_y_binarized))]
pred_y_binarized = lr_oracle.predict(test_xo)
pred_y_binarized = [1 if pred_y_binarized[i] > 1 else 0 for i in range(len(pred_y_binarized))]

lr_train_acc = metrics.accuracy_score(train_yo, lr_oracle.predict(train_xo))
lr_test_acc = metrics.accuracy_score(test_y_binarized, pred_y_binarized)

print ('oracle train acc: ' + str(lr_train_acc))
print ('oracle test acc: ' + str(lr_test_acc))
print ('oracle precision: ' + str(metrics.precision_score(test_y_binarized, pred_y_binarized, average = 'binary')))
print ('oracle recall: ' + str(metrics.recall_score(test_y_binarized, pred_y_binarized, average = 'binary')))